In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2025 -  United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#  Authored by:    Laura Murgatroyd (STFC-UKRI)
#                  Franck Vidal (STFC-UKRI)
#                  Gemma Fardell (STFC-UKRI)

# Flexible Geometry

This notebook introduces the `Cone3D_Flex` `AcquisitionGeometry` which allows setting a different source and detector position for each acquired radiograph.

Learning objectives:
- Create a `Cone3D_Flex` `AcquisitionGeometry`
- Reconstruct using FDK from ASTRA
- Compare the forward projections to the radiographs
- Reconstruct using SIRT

In [ ]:
from cil.io import TIFFStackReader
from cil.utilities.display import show2D, show_system_positions, show_geometry
from cil.framework import ImageGeometry, AcquisitionGeometry, AcquisitionData
import os
from cil.utilities.jupyter import islicer
import matplotlib
import numpy as np
from cil.plugins.astra import FBP
from cil.processors import TransmissionAbsorptionConverter

## Load the Radiographs

Here we use a simulated dataset available on [zenodo](https://zenodo.org/records/16919651) which can be downloaded from: https://zenodo.org/records/16919651/files/non-standard_dragon.zip.

If you are running this notebook locally you will need to download the data and change the path in the following cell. 

In [ ]:
filepath=r"/mnt/materials/SIRF/Fully3D/CIL/non-standard_dragon/projections"


The data consists of TIFF files - the projections, and a CSV file which contains the source and detector positions for each angle

First let's load and view the projections

In [ ]:
projection_array = TIFFStackReader(filepath).read()

In [ ]:
show2D([projection_array]*2, slice_list=[0,100])

In [ ]:
print(projection_array.shape)

### Display the Sinogram

In [ ]:
show2D([projection_array], slice_list=(1,80), title='Projection Array')

The sinogram looks strange! Let's examine the projections using islicer - do you see anything unusual?

In [ ]:
islicer(projection_array, title='Projection Array')

It looks like the dragon is bouncing up and down! This is because some or all of the following are varying between each radiograph:
- detector position
- detector angle
- source position

The data came with a CSV file which describes this geometry for each radiograph.

# Read the Geometry Information

Now we'll read the information from the csv file:

In [ ]:
csv_filepath = os.path.join(filepath, 'geom.csv')

column_names = ["fname",
    "source position (x)", "source position (y)", "source position (z)",
    "imager centre (x)", "imager centre (y)", "imager centre (z)",
    "imager u vector (x)", "imager u vector (y)", "imager u vector (z)",
    "imager v vector (x)", "imager v vector (y)", "imager v vector (z)",
]

# read the csv file:
import pandas as pd
df = pd.read_csv(csv_filepath)

source_position_set = df[['source position (x)', 'source position (y)', 'source position (z)']].values
detector_position_set = df[['imager centre (x)', 'imager centre (y)', 'imager centre (z)']].values
detector_direction_x_set = df[['imager u vector (x)', 'imager u vector (y)', 'imager u vector (z)']].values
detector_direction_y_set = df[['imager v vector (x)', 'imager v vector (y)', 'imager v vector (z)']].values


# Create a CIL Acquisition Geometry

To define the acquisition geometry in CIL, we create a Cone3D_Flex Acquisition Geometry.

This requires us to set:
- `source_position_set` - This is a list of 3D vectors describing the position of the source for each radiograph acquired.
- `detector_position_set` - This is a list of 3D vectors describing the position of the detector for each radiograph acquired.
- `detector_direction_x_set` - This is a list of 3D vectors describing the direction of the detector_x
- `detector_direction_y_set` - This is a list of 3D vectors describing the direction of the detector_y

We have read all of these from the csv in the cell above!

In [ ]:
acq_geometry = AcquisitionGeometry.create_Cone3D_Flex(source_position_set, detector_position_set,
                                                detector_direction_x_set, detector_direction_y_set)


Note: we could also have set the `volume_centre_position`. This is a 3D vector describing the position of the centre of the reconstructed volume (x,y,z). We have not set this, which means it will be set to the default of [0,0,0]: the origin.

As with other geometry types in CIL, we also need to set the panel size, pixel size, and data labels. Printing the data shape and examining the radiographs we showed above helps us with this:

In [ ]:
print(projection_array.shape)

We can see that we have 500 projections and our panel is 160x160

In [ ]:
number_of_pixels = projection_array.shape[1:3]

The pixel size is given by the norms of the detector_direction_x vectors and detector_direction_y vectors:

In [ ]:
pixel_size = [np.linalg.norm(detector_direction_x_set[0]), np.linalg.norm(detector_direction_y_set[0])]
print("Pixel size: ", pixel_size)

In [ ]:
acq_geometry.set_panel(number_of_pixels, pixel_size)


acq_geometry.set_labels(['projection','vertical','horizontal'])

The standard CIL coordinate system is right-handed, and [00_CIL_geometry.ipynb](../1_Introduction/00_CIL_geometry.ipynb) details the specifics.
The coordinate system is aligned with the voxel grid. The centre of the reconstruction is on the origin unless you offset it in the Image Geometry (see notebook 0):

The standard CIL definitions of axes are shown in this image:

![title](images/07_cone_geometry_example.png)

This display tool allows us to visualise the motion of the source and detector throughout the scan:

In [ ]:
show_system_positions(acq_geometry);

Note that in a standard CT scan, the system positions would look something like this:

![title1](images/07_standard_geometry_positions.png)

If we print the acquisition geometry, we can also see the first few source and detector positions, and detector vectors:

In [ ]:
print(acq_geometry)

Note that the `show_geometry` display utility can't be used with Cone_Flex geometry, as it is likely to have a different geometry for each projection!

# Reconstruct with ASTRA

Now we can create our CIL AcquisitionData as normal. We'll be reconstructing with ASTRA so we also reorder the data to enable this:

In [ ]:
acq_data = AcquisitionData(projection_array, geometry=acq_geometry)
acq_data.reorder(order='astra')

In [ ]:
print(acq_data)

We need to convert to absorption data first:

In [ ]:
absorp_data = TransmissionAbsorptionConverter()(acq_data)

In [ ]:
show2D(absorp_data, slice_list=('projection', 1))

Reconstructing requires setting an `ImageGeometry`. This is the description of the reconstruction volume.

First we retrieve the magnification. For `Cone3D_Flex` geometry this is a list containing the magnification for each radiograph. We print the central 10 values as an example to show that the magnification varies in different radiographs in this acquisition.

In [ ]:
mag = absorp_data.geometry.magnification

print("Central 10 values of magnification: ", mag[245:255])

We will scale the pixel size by the mean magnification to calculate the voxel size:

In [ ]:
mean_mag = np.mean(mag)

print("Mean magnification: ", mean_mag)

num_voxel_xy = int(np.ceil(absorp_data.geometry.config.panel.num_pixels[0]))
voxel_size_xy = absorp_data.geometry.config.panel.pixel_size[0] / mean_mag


num_voxel_z = int(np.ceil(absorp_data.geometry.config.panel.num_pixels[1]))
voxel_size_z = absorp_data.geometry.config.panel.pixel_size[1] / mean_mag


image_geometry = ImageGeometry(num_voxel_xy, num_voxel_xy, num_voxel_z, voxel_size_xy, voxel_size_xy, voxel_size_z)

In [ ]:
print(image_geometry)

In [ ]:
fbp = FBP(image_geometry, absorp_data.geometry) 
fbp.set_input(absorp_data)

In [ ]:
recon = fbp.get_output()

In [ ]:
show2D(recon, title='Reconstruction')

We see streaks in the reconstruction.

FDK is an algorithm designed to reconstruct cone-beam CT data acquired from a circular trajectory. The further the acquisition geometry deviates from this ideal circular path, the less defined the reconstruction becomes. In this case, we applied FDK naively to data acquired with a non-standard geometry. While the ray-tracing step in our implementation can accommodate this geometry, the filtering and weighting assumptions of FDK no longer hold, leading to noticeable degradation of the reconstruction.

Here we take a look at a couple of forward projections to check they're as we expect:

In [ ]:

from cil.plugins.astra import ProjectionOperator


PO = ProjectionOperator( recon.geometry, absorp_data.geometry)

forward_projection = PO.direct(recon)

In [ ]:
show2D([forward_projection.array[:,100,:], absorp_data.array[:,100,:]], title=["Forward Proj 100", "Proj 100"], fix_range=(-0.02, 0.9))
show2D([forward_projection.array[:,10,:], absorp_data.array[:,10,:]], title=["Forward Proj 10", "Proj 10"], fix_range=(-0.02, 0.9))

There are different ways to address the issue of degraded reconstruction with FDK:

If the deviations are only in detector position/directions, we could reinterpret the data on to a virtual detector that conforms to an ideal circular geometry. Once transformed, standard FDK can be applied more appropriately. However, this approach may introduce interpolation artifacts.

Alternatively, we could explore modified FDK variants, which will adapt the filtering and weighting steps to better suit non-circular trajectories, potentially improving reconstruction fidelity without fully switching to iterative methods.

Here we will demonstrate using an iterative reconstruction algorithm (in this case SIRT). Iterative reconstruction algorithms are more flexible and can handle arbitrary geometries by incorporating accurate system models that do not rely on standard filtering and weighting.

# SIRT

Here we run the SIRT algorithm, an algebraic iterative method for a weighted least-squares problem. SIRT also accepts certain convex constraints - here we use a non-negativity constraint.

For more information and examples using SIRT, please see [04_FBP_CGLS_SIRT.ipynb](../1_Introduction/04_FBP_CGLS_SIRT.ipynb) and [05_usb_limited_angle_fbp_sirt.ipynb](../1_Introduction/05_usb_limited_angle_fbp_sirt.ipynb).

In [ ]:
from cil.optimisation.algorithms import SIRT

x0 = recon.geometry.copy().allocate(0)

sirt = SIRT(initial=x0, operator=PO, data=absorp_data, lower=0)
sirt.update_objective_interval = 10
sirt.run(200)

recon_sirt = sirt.solution

We can see we're reaching convergence!

In [ ]:
from matplotlib import pyplot as plt
plt.plot(range(0,210,10), sirt.objective,  marker='+');
plt.yscale('log');
plt.xlabel('Iteration');
plt.ylabel('Objective value');

In [ ]:
show2D([recon, recon_sirt], title=['FBP Reconstruction', 'SIRT Reconstruction'], fix_range=(-0.025, 0.2));